### Dense Retrieval

In [2]:
!pip install cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [14]:
import cohere
import numpy as np
import pandas as pd
from tqdm import tqdm

api_key = 'redacted'
co = cohere.Client(api_key)

In [6]:
text = """
Interstellar is a 2014 epic science fiction drama film directed by Christopher Nolan,
who co-wrote the screenplay with his brother Jonathan Nolan. It stars Matthew McConaughey,
Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, and Michael Caine. Set in a
dystopian future where Earth is suffering from catastrophic blight and famine, the film
follows a group of astronauts who travel through a wormhole near Saturn in search of a
new home for mankind.

The screenplay had its origins in a script Jonathan developed in 2007 and was originally
set to be directed by Steven Spielberg. Theoretical physicist Kip Thorne was an executive
producer and scientific consultant on the film, and wrote the tie-in book The Science of Interstellar.
It was Lynda Obst's final film as producer before her death. Cinematographer Hoyte van Hoytema
shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm. Filming began
in late 2013 and took place in Alberta, Klaustur, and Los Angeles. Interstellar uses extensive
practical and miniature effects, and the company DNEG created additional digital effects.

Interstellar was released in theaters on November 7, 2014. In the United States, it was first
released on film stock, expanding to venues using digital projectors. The film received generally
positive reviews and grossed $681 million worldwide during its initial theatrical run, making it the
tenth-highest-grossing film of 2014. Among its various accolades, Interstellar was nominated for five
awards at the 87th Academy Awards, winning Best Visual Effects."""

texts = text.split('.')
texts = [t.strip(' \n') for t in texts]
texts

['Interstellar is a 2014 epic science fiction drama film directed by Christopher Nolan,\nwho co-wrote the screenplay with his brother Jonathan Nolan',
 'It stars Matthew McConaughey,\nAnne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, and Michael Caine',
 'Set in a\ndystopian future where Earth is suffering from catastrophic blight and famine, the film\nfollows a group of astronauts who travel through a wormhole near Saturn in search of a\nnew home for mankind',
 'The screenplay had its origins in a script Jonathan developed in 2007 and was originally\nset to be directed by Steven Spielberg',
 'Theoretical physicist Kip Thorne was an executive\nproducer and scientific consultant on the film, and wrote the tie-in book The Science of Interstellar',
 "It was Lynda Obst's final film as producer before her death",
 'Cinematographer Hoyte van Hoytema\nshot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm',
 'Filming began\nin late 2013 and took place in Albe

In [6]:
response = co.embed(
    texts=texts,
    input_type="search_document",
).embeddings

embeds = np.array(response)
print(embeds.shape)

(14, 4096)


In [13]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 18.5 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [14]:
import faiss

dim = embeds.shape[1]
index = faiss.IndexFlatL2(dim)
print(index.is_trained)
index.add(np.float32(embeds))

True


In [15]:
def search(query, number_of_results=3):
    query_embed = co.embed(texts=[query], input_type="search_query").embeddings[0]
    distances, similar_item_ids = index.search(np.float32([query_embed]), number_of_results)
    texts_np = np.array(texts)
    results = pd.DataFrame(data={"texts": texts_np[similar_item_ids[0]], "distance": distances[0]})
    print(f"Query:'{query}'\nNearest neighbors:")
    return results

In [17]:
query = "how precise was the science"
results = search(query)
results

Query:'how precise was the science'
Nearest neighbors:


,texts,distance
0,Interstellar uses extensive\npractical and min...,11968.047852
1,Theoretical physicist Kip Thorne was an execut...,12780.249023
2,Cinematographer Hoyte van Hoytema\nshot it on ...,13188.646484


In [19]:
!pip install rank_bm25


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [43]:
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string

def bm25_tokenizer(text):
    tokenized_doc = []
    for token in text.lower().split():
        token = token.strip(string.punctuation)
        if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
            tokenized_doc.append(token)
    return tokenized_doc

tokenized_corpus = []
for passage in tqdm(texts):
    tokenized_corpus.append(bm25_tokenizer(passage))

bm25 = BM25Okapi(tokenized_corpus)

def keyword_search(query, top_k=3, num_candidates=14):
    print("Input question:", query)
    # BM25 Search
    bm25_scores = bm25.get_scores(bm25_tokenizer(query))
    top_n = np.argpartition(bm25_scores, -num_candidates)[-num_candidates:]
    bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)
    print(f"Top-3 lexical search (BM25) hits")
    for hit in bm25_hits[0:top_k]:
        print("\t{:.3f}\t{}".format(hit['score'], texts[hit['corpus_id']].replace("\n", " ")))

100%|█████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 3264.96it/s]


In [44]:
keyword_search(query = "how precise was the science")

Input question: how precise was the science
Top-3 lexical search (BM25) hits
	1.457	Theoretical physicist Kip Thorne was an executive producer and scientific consultant on the film, and wrote the tie-in book The Science of Interstellar
	1.406	Interstellar is a 2014 epic science fiction drama film directed by Christopher Nolan, who co-wrote the screenplay with his brother Jonathan Nolan
	0.000	It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, and Michael Caine


## Reranking

In [45]:
query = "how precise was the science"
results = co.rerank(query=query, documents=texts, top_n=3, return_documents=True)
results.results

[RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='The film received generally\npositive reviews and grossed $681 million worldwide during its initial theatrical run, making it the\ntenth-highest-grossing film of 2014'), index=11, relevance_score=0.062103126),
 RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='Theoretical physicist Kip Thorne was an executive\nproducer and scientific consultant on the film, and wrote the tie-in book The Science of Interstellar'), index=4, relevance_score=0.026105914),
 RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text="It was Lynda Obst's final film as producer before her death"), index=5, relevance_score=0.010570489)]

In [46]:
for idx, result in enumerate(results.results):
    print(idx, result.relevance_score, result.document.text)

0 0.062103126 The film received generally
positive reviews and grossed $681 million worldwide during its initial theatrical run, making it the
tenth-highest-grossing film of 2014
1 0.026105914 Theoretical physicist Kip Thorne was an executive
producer and scientific consultant on the film, and wrote the tie-in book The Science of Interstellar
2 0.010570489 It was Lynda Obst's final film as producer before her death


In [51]:
# adding reranker with the above keyword method
def keyword_and_reranking_search(query, top_k=3, num_candidates=10):
    print("Input question:", query)

    # BM25 search
    bm25_scores = bm25.get_scores(bm25_tokenizer(query))
    top_n = np.argpartition(bm25_scores, -num_candidates)[-num_candidates:]
    bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)

    print(f"Top-3 lexical search (BM25) hits")
    for hit in bm25_hits[0:top_k]:
        print("\t{:.3f}\t{}".format(hit['score'], texts[hit['corpus_id']].replace("\n", " ")))

    docs = [texts[hit['corpus_id']] for hit in bm25_hits]

    print(f"\nTop-3 hits by rank-API ({len(bm25_hits)} BM25 hits re-ranked)")
    results = co.rerank(query=query, documents=docs, top_n=top_k, return_documents=True)
    for hit in results.results:
        print("\t{:.3f}\t{}".format(hit.relevance_score, hit.document.text.replace("\n", " ")))

In [52]:
keyword_and_reranking_search("how precise was the science")

Input question: how precise was the science
Top-3 lexical search (BM25) hits
	1.457	Theoretical physicist Kip Thorne was an executive producer and scientific consultant on the film, and wrote the tie-in book The Science of Interstellar
	1.406	Interstellar is a 2014 epic science fiction drama film directed by Christopher Nolan, who co-wrote the screenplay with his brother Jonathan Nolan
	0.000	Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm

Top-3 hits by rank-API (10 BM25 hits re-ranked)
	0.026	Theoretical physicist Kip Thorne was an executive producer and scientific consultant on the film, and wrote the tie-in book The Science of Interstellar
	0.011	It was Lynda Obst's final film as producer before her death
	0.004	Set in a dystopian future where Earth is suffering from catastrophic blight and famine, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind


## RAG

In [53]:
query = "income generated"

# using embedding search, but we would normally do a hybrid
results = search(query)
docs_dict = [{'text': text} for text in results['texts']]
response = co.chat(message=query, documents=docs_dict)
print(response.text)

Query:'income generated'
Nearest neighbors:
The film grossed $681 million worldwide during its initial theatrical run.


In [7]:
# local model
from langchain import LlamaCpp

llm = LlamaCpp(
    model_path="Phi-3-mini-4k-instruct-fp16.gguf",
    n_gpu_layers=1,
    max_tokens=500,
    n_ctx=2048,
    seed=42,
    verbose=False
)

llama_new_context_with_model: n_batch is less than GGML_KQ_MASK_PAD - increasing to 32
llama_new_context_with_model: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64           (not supported)
ggml_metal_init: skipping kernel_flash_

In [8]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name='thenlper/gte-small')

In [9]:
from langchain.vectorstores import FAISS
db = FAISS.from_texts(texts, embedding_model)

In [10]:
from langchain import PromptTemplate

template = """<|user|>
Relevant information:
{context}

Provide a concise answer the following question using the relevant information provided above:
{question}<|end|>
<|assistant|>"""
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [12]:
from langchain.chains import RetrievalQA

rag = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=db.as_retriever(),
    chain_type_kwargs={
        "prompt": prompt
    },
    verbose=True
)

In [13]:
rag.invoke('Income generated')



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Income generated',
 'result': ' The interstellar film, shot by cinematographer Hoyte van Hoytema on 35 mm movie film and IMAX 70mm in anamorphic format, generated approximately $681 million worldwide during its initial theatrical run. This made it the tenth-highest-grossing film of 2014.'}